In [14]:
import pandas as pd
import utils
import datetime as dt
from dateutil.parser import *

In [15]:
df_trades = pd.read_pickle("USD_JPY_H4_trades.pkl")

In [16]:
pair = "USD_JPY"

In [17]:
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, "M5"))

In [18]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [19]:
df_trades['time'] = [parse(x) for x in df_trades.time]
df_raw['time'] = [parse(x) for x in df_raw.time]


In [20]:
df_trades["next"] = df_trades["time"].shift(-1)

In [21]:
df_trades['trade_end'] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades['trade_start'] = df_trades.time + dt.timedelta(hours=4)
df_trades['trade_length'] = df_trades['trade_end'] - df_trades['trade_start']

In [22]:
df_trades[['time', 'next', 'trade_end', 'trade_length']].head()

,time,next,trade_end,trade_length
0,2018-01-02 02:00:00+00:00,2018-01-04 02:00:00+00:00,2018-01-04 05:55:00+00:00,1 days 23:55:00
12,2018-01-04 02:00:00+00:00,2018-01-04 22:00:00+00:00,2018-01-05 01:55:00+00:00,0 days 19:55:00
17,2018-01-04 22:00:00+00:00,2018-01-05 14:00:00+00:00,2018-01-05 17:55:00+00:00,0 days 15:55:00
21,2018-01-05 14:00:00+00:00,2018-01-08 14:00:00+00:00,2018-01-08 17:55:00+00:00,2 days 23:55:00
27,2018-01-08 14:00:00+00:00,2018-01-09 18:00:00+00:00,2018-01-09 21:55:00+00:00,1 days 03:55:00


In [10]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [23]:
def signal_text(signal):
    if signal == 1:
        return 'BUY'
    elif signal == -1:
        return 'SELL'
    return 'NONE'

def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    return False

def end_hit_calc(direction, SL, price, start_price):
    delta = price - start_price
    full_delta = start_price - SL
    fraction = abs(delta / full_delta)
   
    if direction == 1 and price >= start_price:
        return fraction
    elif direction == 1 and price < start_price:
        return -fraction
    elif direction == -1 and price <= start_price:
        return fraction
    elif direction == -1 and price > start_price:
        return -fraction

def process_buy(TP, SL, ask_prices, bid_prices, entry_price):
    for index, price in enumerate(ask_prices):
        if triggered(1, price, entry_price) == True:
            for live_price in bid_prices[index:]:
                if live_price >= TP:
                    return 2.0
                elif live_price <= SL:
                    return -1.0
            return end_hit_calc(1, SL, live_price, entry_price)
    return 0.0

def process_sell(TP, SL, ask_prices, bid_prices, entry_price):
    for index, price in enumerate(bid_prices):
        if triggered(-1, price, entry_price) == True:
            for live_price in ask_prices[index:]:
                if live_price <= TP:
                    return 2.0
                elif live_price >= SL:
                    return -1.0
            return end_hit_calc(-1, SL, live_price, entry_price)
    return 0.0

In [24]:
total = 0
for index, row in df_trades.iterrows():
    m5_data = df_raw[(df_raw.time >= row.trade_start) & (df_raw.time <= row.trade_end)]
    if row.SIGNAL == 1:
        r = process_buy(row.TAKEPROFIT, row.STOPLOSS, m5_data.ask_c.values, m5_data.bid_c.values, row.ENTRY)
        total += r
    else:
        r = process_sell(row.TAKEPROFIT, row.STOPLOSS, m5_data.ask_c.values, m5_data.bid_c.values, row.ENTRY)
        total += r
print(total)

43.36120153064007
